In [1]:
import json
import rppglib.train
import rppglib.models
import torch
from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys
import time

In [2]:
test_dataset = 'mcd_rppg_side'

In [3]:
def open_cfg(file):
    with open(file, 'rt') as f:
        data = json.loads(f.read())
    return data

In [4]:
cfgs = sorted(glob('results/*.cfg'))
print(len(cfgs))

95


In [5]:
class config:
    pass

In [6]:
test = 10
for cfg_file in cfgs:
    print('Loading config:', cfg_file)
    cfg = open_cfg(cfg_file)
    for k, v in cfg.items():
        setattr(config, k, v)

    config.device = 'cuda:0'
    model = rppglib.models.__dict__[config.model](config)
    print(f'Model {config.model} loaded')    

    df = pd.read_csv(f'{test_dataset}.csv')
    print('Test files:',df.shape[0])
    test_ds = rppglib.train.rPPG_Dataset(df['file'].values, model.video_processing, config)
    test_dl = torch.utils.data.DataLoader(test_ds, batch_size=1, num_workers=1, shuffle=False)


    true_ppgs_epoch = list()
    pred_ppgs_epoch = list()
    i = 0
    start = time.time()
    for videos, true_ppgs in tqdm(test_dl, desc='Testing', file=sys.stdout):
        i += 1
        if i > test:
            break
            
        assert not torch.isnan(videos).any(), 'NaNs in Videos'
        assert not torch.isnan(true_ppgs).any(), 'NaNs in PPGs'
        pred_ppgs = model.predict(videos)
        assert not torch.isnan(pred_ppgs).any(), 'NaNs in pred PPGs'
        
        #ppg_mae, hr_mae = clac_metrics(true_ppgs, pred_ppgs)
        #ppg_maes += ppg_mae
        #hr_maes += hr_mae
        
        # true_ppgs_epoch.append(true_ppgs.detach().cpu().numpy())
        # pred_ppgs_epoch.append(pred_ppgs.detach().cpu().numpy())

    config.inference_time_cuda = (time.time() - start)/10
    # true_ppgs_epoch = np.concatenate(true_ppgs_epoch, axis=0)
    # pred_ppgs_epoch = np.concatenate(pred_ppgs_epoch, axis=0)


    # true_ppgs_epoch = true_ppgs_epoch[:, :pred_ppgs_epoch.shape[1]]
    # print('Testing true PPGs:', true_ppgs_epoch.shape)
    # print('Testing pred PPGs:', pred_ppgs_epoch.shape)

    # ppg_mae, hr_mae = rppglib.train.calc_metrics(true_ppgs_epoch, pred_ppgs_epoch)
    
    # config.test_results[f'test__{test_dataset}__ppg'] = ppg_mae
    # config.test_results[f'test__{test_dataset}__hr'] = hr_mae
    # print('PPG MAE:', ppg_mae)
    # print('HR MAE:', hr_mae)

    cfg = {k:v for k, v in config.__dict__.items() if not k.startswith('__')}
    with open(cfg_file, 'w') as fp:
        json.dump(cfg, fp)

Loading config: results/OMIT__mcd_rppg__1.cfg
Model OMIT loaded
Test files: 318
Testing:   3%|███▋                                                                                                                 | 10/318 [00:01<00:36,  8.44it/s]
Loading config: results/PBV__mcd_rppg__1.cfg
Model PBV loaded
Test files: 318
Testing:   3%|███▋                                                                                                                 | 10/318 [00:01<00:38,  8.04it/s]
Loading config: results/POS__mcd_rppg__1.cfg
Model POS loaded
Test files: 318
Testing:   3%|███▋                                                                                                                 | 10/318 [00:02<01:01,  4.97it/s]
Loading config: results/PhysFormer__MMPD__1.cfg
Model PhysFormer loaded
Test files: 318
Testing:   3%|███▋                                                                                                                 | 10/318 [00:21<11:14,  2.19s/it]
Loading config: 

KeyError: 'myPOS'

In [ ]:
1

In [ ]:
1